# Saving Data to External Files
This notebook gives examples of how to write out selected data from [`GBTFitsLoad`](https://dysh.readthedocs.io/en/latest/modules/dysh.fits.html#module-dysh.fits.gbtfitsload) and how to save 
[`Spectrum`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#module-dysh.spectra.spectrum), 
[`Scan`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#module-dysh.spectra.scan), and 
[`ScanBlock`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.scan.ScanBlock) to different formats.

You can find a copy of this tutorial as a Jupyter notebook [here](https://github.com/GreenBankObservatory/dysh/blob/main/notebooks/examples/dataIO.ipynb) or download it by right clicking  <a href="https://raw.githubusercontent.com/GreenBankObservatory/dysh/refs/heads/main/notebooks/examples/dataIO.ipynb" download>here</a> and selecting "Save Link As".

## Loading Modules
We start by loading the modules we will use for this example. 

For display purposes, we use the static (non-interactive) matplotlib backend in this tutorial. However, you can tell `matplotlib` to use the `ipympl` backend to enable interactive plots. This is only needed if working on jupyter lab or notebook.

In [ ]:
# Set interactive plots in jupyter.
#%matplotlib ipympl

# These modules are required for loading and reading data.
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.spectra.spectrum import Spectrum

# We will use matplotlib for plotting.
import matplotlib.pyplot as plt

# These modules are only used to download the data.
from pathlib import Path
from dysh.util.download import from_url

## Data Retrieval

Download the example SDFITS data, if necessary.

In [ ]:
url = "http://www.gb.nrao.edu/dysh/example_data/positionswitch/data/AGBT05B_047_01/AGBT05B_047_01.raw.acs/AGBT05B_047_01.raw.acs.fits"
savepath = Path.cwd() / "data"
savepath.mkdir(exist_ok=True) # Create the data directory if it does not exist.
filename = from_url(url, savepath)

## Data Loading

Next, we use `GBTFITSLoad` to load the data, and then its `summary` method to inspect its contents.

In [ ]:
sdfits = GBTFITSLoad(filename)
sdfits.summary()

### Calibrate a position switched scan.  

This returns a `ScanBlock` containing one [`PSScan`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.scan.PSScan) with 11 integrations for the ON position and 11 integrations for the OFF position.

In [ ]:
ps_scan_block = sdfits.getps(scan=51, ifnum=0, plnum=0, fdnum=0)

In [ ]:
print(f"Number of integrations = {ps_scan_block[0].nrows}")

Get the time average of the calibrated data. This method returns a `Spectrum`.

In [ ]:
ta = ps_scan_block.timeaverage()

In [ ]:
ta.plot()

## Reading and Writing Individual Spectra

###  Inputs and Outputs
`dysh` supports output to text files in a variety of [formats familiar to users of astropy](https://docs.astropy.org/en/stable/io/ascii/index.html#id1):
* basic
* commented_header
* ECSV
* fixed_width
* IPAC
* MRT
* votable

The following lines of code define some of the available formats in a list, and then loop over them saving the calibrated data in each format.
We use the `overwrite=True` parameter to avoid errors if the files already exist on disk.

In [ ]:
# Define the formats in a list.
fmt = [ 
    "basic",
    "commented_header",
    "ecsv",
    "fixed_width",
    "ipac",
    "mrt",
    "votable",
]

# Define the output directory and create it if it does not exists already.
output_dir = Path.cwd() / "output"
output_dir.mkdir(exist_ok=True)

# Loop over formats writing the calibrated spectrum.
for f in fmt:
    file = output_dir / f"testwrite.{f}"
    ta.write(file, format=f, overwrite=True)

We can also write a `Spectrum` to FITS format.

In [ ]:
ta.write(output_dir / "testwrite.fits", format="fits", overwrite=True)

We can read spectra in FITS and a few formats. [As noted in astropy, ECSV ](https://docs.astropy.org/en/stable/io/ascii/ecsv.html#ecsv-format) is the only ASCII format that can make a lossless output-input roundtrip and thus reproduce an original spectrum.

We use the `Spectrum.read` method to read the saved data.

In [ ]:
s1 = Spectrum.read(output_dir / "testwrite.fits", format="fits")
s1.plot()

In [ ]:
s2 = Spectrum.read(output_dir / "testwrite.ecsv", format="ecsv")
s2.plot()

### GBTIDL ASCII format
`dysh` can read text files created by GBTIDL's `write_ascii` function. However, those files do not provide sufficient metadata to fully recreate the spectrum.  (For instance, they do not have complete sky coordinate information.)  

In [ ]:
url = "https://www.gb.nrao.edu/dysh/example_data/onoff-L/gbtidl-data/onoff-L_gettp_156_intnum_0_HEL.ascii"
filename_ascii = from_url(url, savepath)
s3 = Spectrum.read(filename_ascii, format='gbtidl')

In [ ]:
print(s3, "\n", s3.meta)

`dysh` can even read compressed ASCII files. Note these data have velocity on the spectral axis.

In [ ]:
url = "https://www.gb.nrao.edu/dysh/example_data/onoff-L/gbtidl-data/onoff-L_getps_152_RADI-HEL.ascii.gz"
filename_ascii_gz = from_url(url, savepath)
s4 = Spectrum.read(filename_ascii_gz, format='gbtidl')

In [ ]:
print(s4, "\n", s4.meta)

#### Plot 
To plot the spectrum contained in the ascii files you have to use `matplotlib`.

In [ ]:
plt.figure()
plt.xlabel(f"Velocity ({s4.meta['VELDEF']}) [{s4.spectral_axis.unit}]")
plt.ylabel(f"$T_A$ [{s4.flux.unit}]")
plt.plot(s4.spectral_axis, s4.flux)

## Writing Multiple Calibrated Spectra to SDFITS
You can write the calibrated data from a `ScanBlock` to the SDFITS format.
If there are multiple scans in the `ScanBlock`, they will all be written to the same SDFITS (useful for `gbtgridder`).

In [ ]:
ps_scan_block.write(output_dir / "scanblock.fits", overwrite=True)

## Reading Calibrated SDFITS

To load the saved data, we use the same function we used to load the raw data, `GBTFITSLoad`.

In [ ]:
ps_scan_block_read = GBTFITSLoad(output_dir / "scanblock.fits")

This is treated the same was as the raw data, so the same methods are available.

In [ ]:
ps_scan_block_read.summary()

However, since the data is already calibrated, trying to fetch the data using calibration methods like `getps` or `getnod` will result in errors.
Instead, we can use the `gettp` method or `getspec`.

### Using `gettp`

If we use `gettp`, then we will get all of the integrations as a `ScanBlock` object.

In [ ]:
tp_read = ps_scan_block_read.gettp(scan=51, ifnum=0, plnum=0, fdnum=0)
print(f"Number of integrations: {tp_read[0].nint}")

We can access individual integration through the `calibrated` method.

In [ ]:
ta_read_0a = tp_read[0].getspec(0)
ta_read_0a.plot()

### Using `getspec`

Now we do the same using `getspec`. This method takes as input the row number we want to retrieve.

In [ ]:
ta_read_0b = ps_scan_block_read.getspec(0)

In [ ]:
ta_read_0b.plot()

If we take the difference, it is zero.

In [ ]:
(ta_read_0a.data - ta_read_0b.data).sum()

## Writing Out Selected Data from `GBTFITSLoad`
The `write()` method of `GBTFITSLoad` supports down-selection of data. 
Data can be selected on any SDFITS column.

In the following call to `GBTFITSLoad.write` we will select a single polarization (`plnum=1`), and only the first five integrations of each scan (`intnum=range(5)`). We also set `overwrite=True` to avoid errors if the file already exists, and request that the output be saved into a single file (`multifile=False`).

In [ ]:
sdfits.write(output_dir / "mydata.fits", plnum=1, 
             intnum=range(5), overwrite=True, multifile=False)

These data, can of course, be read back in.

In [ ]:
sdfits2 = GBTFITSLoad(output_dir / "mydata.fits")

In [ ]:
sdfits2.summary()

### Writing SDFITS to Multiple Files 
If the data came from multiple files, for instance VEGAS banks, then by default they are written to multiple files, so
`sdfits.write('mydata.fits')`
would write to mydata0.fits, mydata1.fits, ... mydataN.fits.
